In [ ]:
from datetime import datetime

import akshare as ak  # 升级到最新版
import backtrader as bt  # 升级到最新版
import matplotlib.pyplot as plt  # 由于 Backtrader 的问题，此处要求 pip install matplotlib==3.2.2
import pandas as pd
import pyfolio as pf
import quantstats as qs
import platform
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import font_manager

def set_chinese_font():
    system = platform.system()  # 获取操作系统名称
    if system == "Windows":
        # Windows 使用 SimHei
        rcParams['font.sans-serif'] = ['STXihei']
    elif system == "Darwin":
        # macOS 使用 PingFang SC
        rcParams['font.sans-serif'] = ['Heiti TC']
    else:
        # 其他系统提示用户自行安装字体
        raise EnvironmentError("Unsupported OS. Please install a suitable Chinese font.")

    # 解决负号显示问题
    rcParams['axes.unicode_minus'] = False

# 设置字体
set_chinese_font()
# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="600256", adjust="hfq").iloc[:, :7]
# 删除 `股票代码` 列
del stock_hfq_df['股票代码']
# 处理字段命名，以符合 Backtrader 的要求
stock_hfq_df.columns = [
    "date",
    "open",
    "close",
    "high",
    "low",
    "volume",
]
# 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df["date"])

class MyStrategy(bt.Strategy):
    """
    主策略程序
    """
    # 设定交易策略的参数
    params = dict(
        maperiod=20,
    )
    def __init__(self):
        """
        初始化函数
        """
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buy_price = None
        self.buy_comm = None
        # 添加移动均线指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0].lines.close, period=self.params.maperiod
        )
        self.sma.csv = True

    def next(self):
        """
        执行逻辑
        """
        if self.order:  # 检查是否有指令等待执行，如果有未完成的订单则跳过
            return
        # 检查是否持仓
        if not self.position:  # 没有持仓
            # 执行买入条件判断：收盘价格上涨突破 20 日均线
            if self.datas[0].lines.close[-1] < self.sma[-1] and self.datas[0].lines.close[0] > self.sma[0]:
                self.order = self.buy(size=100)  # 执行买入
        # 执行卖出条件判断：收盘价格跌破 20 日均线
        elif self.datas[0].lines.close[-1] > self.sma[-1] and self.datas[0].lines.close[0] < self.sma[0]:
            self.order = self.sell(size=100)  # 执行卖出


In [ ]:
cerebro = bt.Cerebro()  # 初始化回测系统
start_date = datetime(2023, 4, 3)  # 回测开始时间
end_date = datetime(2024, 9, 12)  # 回测结束时间
data = bt.feeds.PandasData(
    dataname=stock_hfq_df, fromdate=start_date, todate=end_date
)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统
cerebro.addstrategy(MyStrategy)  # 将交易策略加载到回测系统中

# 添加观察者 PyFolio
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
# 添加书写者
cerebro.addwriter(bt.WriterFile, csv=True, out="result.csv")

start_cash = 1000000
cerebro.broker.setcash(start_cash)  # 设置初始资本为 100000
cerebro.broker.setcommission(commission=0.002)  # 设置交易手续费为 0.2%
result = cerebro.run()[0]  # 运行回测系统

port_value = cerebro.broker.getvalue()  # 获取回测结束后的总资金
pnl = port_value - start_cash  # 盈亏统计

print(
    f"初始资金: {start_cash}\n回测期间：{start_date.strftime('%Y%m%d')}:{end_date.strftime('%Y%m%d')}"
)
print(f"总资金: {round(port_value, 2)}")
print(f"净收益: {round(pnl, 2)}")

# cerebro.plot(style="candlestick")  # 画图

portfolio_stats = result.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
print(returns)
pf.create_full_tear_sheet(returns)
qs.reports.full(returns)

qs.stats.sharpe(returns)
qs.stats.conditional_value_at_risk(returns)

help(qs.stats.conditional_value_at_risk)
qs.extend_pandas()
stock = qs.utils.download_returns('000001.SS')
stock = stock[str(start_date):str(end_date)]
stock.sharpe()
qs.plots.snapshot(stock['000001.SS'], title='000001.SS')
returns.index = returns.index.tz_convert(None)
stock['000001.SS'].index=stock['000001.SS'].index.tz_convert(None)
qs.reports.html(returns, benchmark=stock['000001.SS'],benchmark_title="Facebook", output='stats.html', title='Report')  # 会在本地目录生成 html 文件，用游览器打开即可


In [ ]:
import matplotlib
from matplotlib import font_manager
 
font_list=sorted([f.name for f in matplotlib.font_manager.fontManager.ttflist])
for i in font_list:
    print(i)

In [ ]:
from datetime import datetime,date

def generate_quarters(today):
    """
    动态生成从三年前最近的季度末到当前季度末的所有季度时间点
    """
    # 计算当前日期最近的季度末日期
    if today.month in [1, 2, 3]:
        end_date = date(today.year, 3, 31)
    elif today.month in [4, 5, 6]:
        end_date = date(today.year, 6, 30)
    elif today.month in [7, 8, 9]:
        end_date = date(today.year, 9, 30)
    else:  # 10, 11, 12
        end_date = date(today.year, 12, 31)

    # 计算三年前最近的季度末日期
    start_date = end_date.replace(year=end_date.year - 3)

    # 存储季度列表
    quarters = []
    current_date = start_date

    # 生成从 start_date 到 end_date 的季度列表
    while current_date <= end_date:
        if current_date.month in [1, 2, 3]:
            quarter_end = f"{current_date.year}0331"
        elif current_date.month in [4, 5, 6]:
            quarter_end = f"{current_date.year}0630"
        elif current_date.month in [7, 8, 9]:
            quarter_end = f"{current_date.year}0930"
        elif current_date.month in [10, 11, 12]:
            quarter_end = f"{current_date.year}1231"

        quarters.append(quarter_end)

        # 跳到下一个季度
        if current_date.month in [1, 2, 3]:
            current_date = date(current_date.year, 4, 1)
        elif current_date.month in [4, 5, 6]:
            current_date = date(current_date.year, 7, 1)
        elif current_date.month in [7, 8, 9]:
            current_date = date(current_date.year, 10, 1)
        elif current_date.month in [10, 11, 12]:
            current_date = date(current_date.year + 1, 1, 1)

    return quarters

In [126]:
from datetime import timedelta,datetime
import akshare as ak
# 根据当前时间,往前推四个季度,df合并,然后筛选股权登记日在开始时间和结束时间内的
# 调用函数生成季度时间点
    # 获取当前日期
end_date_str = "2024-11-18"
# 转换为 datetime 对象
end_date_obj = datetime.strptime(end_date_str, "%Y-%m-%d")- timedelta(days=1)
quarters = generate_quarters(end_date_obj)
start_date_obj = end_date_obj - timedelta(days=365)

df=pd.DataFrame()
for tt in quarters:
    df=pd.concat([df,ak.stock_fhps_em(date=tt)], ignore_index=True)
# 转换股权登记日为 datetime 格式
df["股权登记日"] = pd.to_datetime(df["股权登记日"])
# 筛选股权登记日在范围内的记录
print(f"start:{start_date_obj}")
print(f"end:{end_date_obj}")
filtered_df = df[(df["股权登记日"] >= pd.Timestamp(start_date)) & (df["股权登记日"] <= pd.Timestamp(end_date))]
result_df = filtered_df[["代码","现金分红-现金分红比例","现金分红-股息率"]].groupby(['代码']).sum().sort_values(by='现金分红-股息率', ascending=False).reset_index()
# 从大到小排的股息
result_df

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

start:2023-11-18 00:00:00
end:2024-11-17 00:00:00


,代码,现金分红-现金分红比例,现金分红-股息率
0,002582,10.0,0.164501
1,002303,4.5,0.160173
2,000517,3.2,0.120755
3,603897,22.0,0.109311
4,873679,13.0,0.098499
...,...,...,...
3923,003026,0.0,0.000000
3924,301037,0.0,0.000000
3925,603155,0.0,0.000000
3926,688598,0.0,0.000000


In [125]:
df[df.代码=="001368"]


,代码,名称,送转股份-送转总比例,送转股份-送转比例,送转股份-转股比例,现金分红-现金分红比例,现金分红-股息率,每股收益,每股净资产,每股公积金,每股未分配利润,净利润同比增长,总股本,预案公告日,股权登记日,除权除息日,方案进度,最新公告日期
4640,001368,通达创智,NaN,NaN,NaN,2.0,0.007965,1.57,8.391734,3.939764,3.037073,0.221037,112000000,2023-04-28,2023-05-31,2023-06-01,实施分配,2023-05-25
7397,001368,通达创智,NaN,NaN,NaN,8.0,0.037523,0.96,12.569543,8.276328,2.900093,-23.183556,113867600,2024-03-28,2024-04-26,2024-04-29,实施分配,2024-04-20
12134,001368,通达创智,NaN,NaN,NaN,6.5,0.030531,0.74,12.423582,8.418372,2.789255,3.603177,113867600,2024-08-29,2024-11-25,2024-11-26,实施分配,2024-11-20


,代码,现金分红-现金分红比例,现金分红-股息率
7252,601919,5.1,0.048159
9487,601919,2.3,0.014763


In [ ]:
stock_fhps_em_df[stock_fhps_em_df.代码=="002582"]
stock_fhps_em_df